# 🌌 Introducción al Clustering con Exoplanetas (Día 1)

### Enunciado de la actividad
La misión científica **"Catálogo Exoplanetario"** de la NASA quiere entender mejor la diversidad de mundos descubiertos fuera del Sistema Solar.

El equipo de datos ha recibido un **catálogo real de exoplanetas confirmados**. En este catálogo aparecen, entre otros, datos como:

- Periodo orbital del planeta (días) `pl_orbper`
- Masa del planeta en masas de Júpiter `pl_bmassj`
- Radio del planeta en radios de Júpiter `pl_radj`
- Temperatura efectiva de la estrella `st_teff`
- Masa de la estrella `st_mass`

No existe una etiqueta oficial de "tipo de planeta" (tipo Tierra, tipo Neptuno, tipo Júpiter caliente, etc.),
así que vamos a usar **clustering no supervisado** para descubrir **grupos naturales**.

Tu objetivo como miembro de la tripulación será:
1. Explorar visualmente el catálogo de exoplanetas.
2. Aplicar un algoritmo de **clustering jerárquico (Agglomerative)** para agrupar planetas por similitud.
3. Interpretar qué tipo de planetas parece haber en cada grupo.

https://dataherb.github.io/flora/nasa_exoplanet_archive/

## 1. Preparación del entorno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D

# Estilo de gráficas
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 2. Carga del catálogo de exoplanetas

Vamos a usar el fichero `nasa_confirmed_exoplanets.csv`, basado en el **NASA Exoplanet Archive**.
Colócalo en la **misma carpeta** que este notebook.

Si no se encuentra el archivo local, el notebook intentará descargar una copia pública compatible.

In [ ]:
local_filename = 'confirmed_exoplanets.csv'
url_backup = 'https://raw.githubusercontent.com/InterImm/nasa-exoplanet-archive/master/dataset/confirmed_exoplanets.csv'

try:
    df = pd.read_csv(local_filename)
    origen = f'archivo local: {local_filename}'
except FileNotFoundError:
    print(f'⚠️ No se ha encontrado el archivo local `{local_filename}`. Intentando descargar desde la URL...')
    df = pd.read_csv(url_backup)
    origen = 'descarga online desde GitHub (NASA Exoplanet Archive mirror)'

print('✅ Datos cargados desde', origen)
print('Número de filas y columnas:', df.shape)
display(df.head())

## 3. Selección de columnas relevantes

Nos vamos a quedar con las columnas que nos interesan para este ejercicio:

- `pl_name`: nombre del planeta
- `pl_orbper`: periodo orbital (días)
- `pl_bmassj`: masa del planeta (en masas de Júpiter)
- `pl_radj`: radio del planeta (en radios de Júpiter)
- `st_teff`: temperatura de la estrella (K)
- `st_mass`: masa de la estrella (en masas solares)


In [ ]:
print('Columnas disponibles en el DataFrame:')
print(df.columns.tolist())

columnas_interes = ['pl_name', 'pl_orbper', 'pl_bmassj', 'pl_radj', 'st_teff', 'st_mass']
columnas_presentes = [c for c in columnas_interes if c in df.columns]
print('\nUsaremos estas columnas (presentes en el dataset):', columnas_presentes)

df_peq = df[columnas_presentes].copy()
display(df_peq.head())

## 4. EDA

En este ejercicio usaremos como base `pl_orbper`, `pl_bmassj` y `pl_radj`.

### 4.1. Visualización de datos

In [ ]:
# Seleccionamos las columnas numéricas que usaremos para el clustering
features_numericas = [c for c in ['pl_orbper', 'pl_bmassj', 'pl_radj'] if c in df_peq.columns]
print('Columnas numéricas elegidas para el clustering:', features_numericas)

# Mostramos cuántos valores nulos hay en cada columna del DataFrame reducido
print("\nValores nulos por columna (df_peq):")
print(df_peq[features_numericas].isna().sum())


# Elige las variables para el gráfico de dispersión
feature_x = 'pl_orbper'
feature_y = 'pl_radj'

if feature_x not in df_peq.columns or feature_y not in df_peq.columns:
    raise ValueError('Revisa feature_x y feature_y: alguna no está en el DataFrame limpio.')

plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_peq,
    x=feature_x,
    y=feature_y,
    alpha=0.6,
    s=35
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(f'{feature_x} (escala log)')
plt.ylabel(f'{feature_y} (escala log)')
plt.title('Exoplanetas: dispersión sin clustering')
plt.tight_layout()
plt.show()

### 4.2 Visualización tras limpieza

In [ ]:
# ======================================================================
# REEMPLAZO DE VALORES NULOS EN VEZ DE ELIMINAR FILAS (dropna)
# ======================================================================

# Hacemos una copia para no modificar df_peq directamente
df_clean = df_peq.copy()

# OPCIÓN 1: Imputar con la MEDIA de cada columna
# ----------------------------------------------
#for col in features_numericas:
    # Calculamos la media de la columna (ignorando NaNs)
#    media_col = df_clean[col].mean()
    
    # Rellenamos los NaN de esa columna con la media calculada
#    df_clean[col] = df_clean[col].fillna(media_col)

# --- Si quisieras imputar con la MEDIANA en lugar de la media, usarías esto ---
for col in features_numericas:
#     # Calculamos la mediana de la columna (ignorando NaNs)
     mediana_col = df_clean[col].median()
     
     # Rellenamos los NaN de esa columna con la mediana calculada
     df_clean[col] = df_clean[col].fillna(mediana_col)
# ------------------------------------------------------------------------------

# Comprobamos que ya no quedan valores nulos en las columnas numéricas elegidas
print("\nValores nulos por columna después del reemplazo (df_clean):")
print(df_clean[features_numericas].isna().sum())

# Mostramos tamaños antes y después (el número de filas se mantiene)
#print('Tamaño tras reemplazar los nulos (df_clean):', df_clean.shape)

# Resumen estadístico de las columnas numéricas tras la imputación
#display(df_clean[features_numericas].describe())

# Elige las variables para el gráfico de dispersión
feature_x = 'pl_orbper'
feature_y = 'pl_radj'

if feature_x not in df_peq.columns or feature_y not in df_peq.columns:
    raise ValueError('Revisa feature_x y feature_y: alguna no está en el DataFrame limpio.')

plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_clean,
    x=feature_x,
    y=feature_y,
    alpha=0.6,
    s=35
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(f'{feature_x} (escala log)')
plt.ylabel(f'{feature_y} (escala log)')
plt.title('Exoplanetas: dispersión sin clustering')
plt.tight_layout()
plt.show()


## 5. Clustering jerárquico (Agglomerative)

Ahora sí aplicamos un algoritmo de clustering **básico / heredado**:
- Usamos **AgglomerativeClustering** (clustering jerárquico aglomerativo).
- Parte de muchos grupos pequeños y los va fusionando según la similitud.

Pasos:
1. Construimos la matriz `X` con las columnas numéricas seleccionadas.
2. Aplicamos `StandardScaler` para que todas las variables tengan peso similar.
3. Ejecutamos `AgglomerativeClustering` para obtener, por ejemplo, 3 clusters.


In [6]:
# Matriz de características numéricas
df_clean = df_peq.dropna(subset=features_numericas).copy()
X = df_clean[features_numericas].values

# Escalado (muy recomendable antes de usar distancias)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Clustering jerárquico aglomerativo
n_clusters = 2  # 🔁 prueba a cambiarlo: 2, 3, 4...
agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
cluster_labels = agg.fit_predict(X_scaled)

df_clean['cluster_hier'] = cluster_labels
print('Nuestros', n_clusters, 'son:')
df_clean['cluster_hier'].value_counts().sort_index()

## 6. Visualización de los grupos

Volvemos al diagrama de dispersión, pero ahora coloreando los puntos según el cluster asignado por el algoritmo jerárquico.

Así podemos ver si los grupos tienen algún sentido físico (por ejemplo, planetas grandes / pequeños, órbitas cortas / largas, etc.).

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_clean,
    x=feature_x,
    y=feature_y,
    hue='cluster_hier',
    palette='tab10',
    alpha=0.7,
    s=35
)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(f'{feature_x} (escala log)')
plt.ylabel(f'{feature_y} (escala log)')
plt.title('Exoplanetas: clustering jerárquico (vista 2D)')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

# 7. Visualización en 3D

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')

# Para que la escala log se vea más clara, convertimos a log10
x = df_clean['pl_orbper']
y = df_clean['pl_radj']
z = df_clean['pl_bmassj']

# Evitamos problemas con valores <= 0 antes del log
mask_pos = (x > 0) & (y > 0) & (z > 0)
x_log = np.log10(x[mask_pos])
y_log = np.log10(y[mask_pos])
z_log = np.log10(z[mask_pos])
clusters = df_clean.loc[mask_pos, 'cluster_hier']

scatter = ax.scatter(
    x_log,
    y_log,
    z_log,
    c=clusters,
    cmap='tab10',
    s=35,
    alpha=0.8
)

ax.set_xlabel('log10(pl_orbper)')
ax.set_ylabel('log10(pl_radj)')
ax.set_zlabel('log10(pl_bmassj)')
ax.set_title('Exoplanetas: clustering jerárquico (vista 3D)')

# Leyenda de clusters
handles, labels = scatter.legend_elements(prop="colors", alpha=0.8)
ax.legend(handles, labels, title="Cluster", loc='upper left')

plt.tight_layout()
plt.show()

### 7.1. Visualización 3D (sin Scaler)

In [ ]:
# ==========================================================
# Visualización 3D del clustering jerárquico (sin log)
# Ejes: pl_orbper, pl_radj, pl_bmassj
# ==========================================================

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')

# Columnas originales
x = df_clean['pl_orbper']
y = df_clean['pl_radj']
z = df_clean['pl_bmassj']

# Por si hubiera algún NaN residual, filtramos filas válidas
mask_valid = (~x.isna()) & (~y.isna()) & (~z.isna())

x_vals = x[mask_valid]
y_vals = y[mask_valid]
z_vals = z[mask_valid]
clusters = df_clean.loc[mask_valid, 'cluster_hier']

scatter = ax.scatter(
    x_vals,
    y_vals,
    z_vals,
    c=clusters,
    cmap='tab10',
    s=35,
    alpha=0.8
)

ax.set_xlabel('pl_orbper (días)')
ax.set_ylabel('pl_radj (radios Júpiter)')
ax.set_zlabel('pl_bmassj (masas Júpiter)')
ax.set_title('Exoplanetas: clustering jerárquico (vista 3D sin log)')

# Leyenda de clusters
handles, labels = scatter.legend_elements(prop="colors", alpha=0.8)
ax.legend(handles, labels, title="Cluster", loc='upper left')

plt.tight_layout()
plt.show()


## 8. Resumen de cada cluster y reflexión

Para interpretar mejor cada grupo, calculamos estadísticas por cluster.


In [ ]:
resumen_clusters = df_clean.groupby('cluster_hier')[features_numericas].median()
resumen_clusters

### Preguntas
---

1. **Interpretación física**
   - ¿Puedes describir cada cluster con frases como:
     - "Cluster 0: planetas pequeños y de periodo corto".
     - "Cluster 1: planetas grandes y lejanos", etc.?
   Si es así, piensa una y ponla aquí:

2. **Número de clusters**
   - Cambia `n_clusters` (2, 3, 4...) y repite el clustering.
   - ¿En qué casos se mezclan demasiado los grupos? ¿En cuál te parece que mejor refleja familias distintas?

3. **Elección de variables**
   - Añade `st_teff` o `st_mass` a `features_numericas` y vuelve a ejecutar.
   - ¿Cómo cambia la forma de los clusters? ¿Tiene sentido que las propiedades de la estrella influyan?

4. **Limitaciones del método**
   - ¿Qué crees que pasa con planetas muy extremos (muy masivos o con periodos orbitales raros)?
   - ¿El algoritmo jerárquico los considera como su propio grupo o los mete a la fuerza en uno existente?